### **ETL in Python with MySQL** 

In [1]:
# Importing dependencies 
import pandas as pd
from sqlalchemy import create_engine
import pymysql

In [2]:
# Reading csv to dataframe
df = pd.read_csv('../Data/data.txt')

# Viewing dataset
df

,Gender,Age,Name
0,M,20,Mike
1,F,54,Lara


In [3]:
def ETL_MySQL(db,tablename,pw):
  """
    :param db: Destination Database name MUST ALREADY EXIST
    :param tablename: Created table name
    :param pw: Enter password
   
    """
  # SQL table header format
  tablename = tablename.lower()
  tablename = tablename.replace(' ','_')

  # engine = create_engine('mysql+pymysql://root:localhost/test', pool_recycle=3306)
  engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host="localhost", db=db, user="root", pw=pw))

  # Convert dataframe to sql table                                   
  df.to_sql('{tb}'.format(tb=tablename), engine, index=False, if_exists='replace')
  return("ETL Successful, {num} rows loaded in table {tb}.".format(num=len(df['Gender']), tb=tablename))


In [4]:
ETL_MySQL("test","THIS IS A TEST","[USER INSTANCE PASSWORD]")

'ETL Successful, 2 rows loaded in table this_is_a_test.'